In [ ]:
import pickle
import shapely
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa
import matplotlib.pyplot as plt

# Add main repo to path
import sys
from os.path import expanduser
sys.path.append(expanduser('/Users/zachkeskinen/Documents/spicy-snow/'))

from spicy_snow.processing.s1_preprocessing import merge_partial_s1_images

from spicy_snow.processing.snow_index import calc_delta_cross_ratio, calc_delta_gamma, \
    clip_delta_gamma_outlier, calc_snow_index, calc_snow_index_to_snow_depth
from spicy_snow.processing.wet_snow import id_newly_wet_snow, id_wet_negative_si, \
    id_newly_frozen_snow, flag_wet_snow

In [ ]:
ds = xr.open_dataset(Path('~/Desktop/spicy-lowman_-117--116_44-45').expanduser())

In [ ]:
merged = merge_partial_s1_images(ds)